In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder\
        .master("spark://192.168.2.97:7077") \
        .appName("Group-6-Question-1-Claude")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.cores.max", 2)\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/11 16:01:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/11 16:01:17 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [25]:
def parse_json_body_to_string(json_filename):
    df = spark.read.json(json_filename)

    my_df_lower = df.select(lower("body").alias("lowercase_body"))

    charachters_to_delete = "[\.,\[\]\(\):_\-!?\'\+=;/&{}@$#*\"\\\\%><|~¨´¤]"

    my_df_clean = my_df_lower.withColumn("lowercase_body", regexp_replace(my_df_lower.lowercase_body, charachters_to_delete, ""))

    my_df_words = my_df_clean.select(explode(split("lowercase_body", "\s+")).alias("word"))

    text_rdd = my_df_words.rdd

    return text_rdd

In [28]:
df = parse_json_body_to_string("../RC_2006-07.json")
df.take(50)

[Row(word='although'),
 Row(word='i'),
 Row(word='normally'),
 Row(word='think'),
 Row(word='all'),
 Row(word='deeply'),
 Row(word='religious'),
 Row(word='people'),
 Row(word='are'),
 Row(word='insane'),
 Row(word='and'),
 Row(word='the'),
 Row(word='moderates'),
 Row(word='lemmings'),
 Row(word='i'),
 Row(word='do'),
 Row(word='like'),
 Row(word='the'),
 Row(word='amish'),
 Row(word='people'),
 Row(word='theyre'),
 Row(word='like'),
 Row(word='a'),
 Row(word='mad'),
 Row(word='scientist'),
 Row(word='dangeroushttpwwwwashingtonpostcomwpdynarticlesa350642004dec29html'),
 Row(word='in'),
 Row(word='real'),
 Row(word='life'),
 Row(word='but'),
 Row(word='very'),
 Row(word='cool'),
 Row(word='to'),
 Row(word='see'),
 Row(word='on'),
 Row(word='tv'),
 Row(word='deleted'),
 Row(word='deleted'),
 Row(word='some'),
 Row(word='friends'),
 Row(word='and'),
 Row(word='i'),
 Row(word='are'),
 Row(word='working'),
 Row(word='on'),
 Row(word='a'),
 Row(word='way'),
 Row(word='of'),
 Row(word='deali

In [31]:
from operator import add

def find_occurances(word_lst):
    #map all words, with value 1
    words_one = word_lst.map(lambda x: (x,1))

    # Add all the ones for every word (by reduceByKey)
    wordcount = words_one.reduceByKey(add)

    # Sort in descending order
    wordcount = wordcount.sortBy(lambda x: x[1], ascending=False)
    
    return wordcount


In [32]:
wc = find_occurances(df)

In [34]:
wc.take(30)

[(Row(word='the'), 67622),
 (Row(word='to'), 39970),
 (Row(word='a'), 33636),
 (Row(word='of'), 31909),
 (Row(word='and'), 28881),
 (Row(word='is'), 24321),
 (Row(word='that'), 23704),
 (Row(word='i'), 22313),
 (Row(word='in'), 20066),
 (Row(word='it'), 17996),
 (Row(word='you'), 16429),
 (Row(word='for'), 12886),
 (Row(word='this'), 10831),
 (Row(word='are'), 10509),
 (Row(word='not'), 10383),
 (Row(word='be'), 10004),
 (Row(word='on'), 9965),
 (Row(word='have'), 9015),
 (Row(word='but'), 8931),
 (Row(word='with'), 8628),
 (Row(word='as'), 8457),
 (Row(word='they'), 8271),
 (Row(word='if'), 7329),
 (Row(word='was'), 6817),
 (Row(word='its'), 6727),
 (Row(word='or'), 6560),
 (Row(word='people'), 5773),
 (Row(word='what'), 5536),
 (Row(word='at'), 5508),
 (Row(word='about'), 5473)]

In [36]:
spark.stop()